# Agentic Explainability Workflow – Usage

This notebook runs the agentic explainability pipeline: ask a natural-language question about optimization results, get a counterfactual run and a summary (trade-offs or infeasibility conflict).

**Prerequisites:**
- `config/secrets.env` with `OPENAI_API_KEY` and `GUROBI_LICENSE_FILE=config/WLS-dev-key.lic`
- Run the first two sections once to create the baseline and RAG index; then you can run only the workflow section.

## 1. Setup paths and load secrets

In [ ]:
import sys
from pathlib import Path

# Project root: directory that contains agentic_explain and data
def find_project_root():
    for start in [Path.cwd()] + list(Path.cwd().parents):
        if (start / "agentic_explain").is_dir() and (start / "data").is_dir():
            return start
    return Path.cwd()

PROJECT_ROOT = find_project_root()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from config.load_secrets import load_secrets, get_gurobi_env_kwargs

load_secrets()  # load from config/secrets.env or project root
print("Project root:", PROJECT_ROOT)

## 2. Run baseline (once)

Saves `outputs/baseline_result.json`, `outputs/model.lp`, `outputs/model.mps`. Skip this cell if you already have them.

In [ ]:
import json
from gurobipy import GRB

from agentic_explain.staffing_model import load_raw_data, process_data, build_gurobi_model

DATA_DIR = PROJECT_ROOT / "data"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

raw = load_raw_data(DATA_DIR)
inputs = process_data(
    raw["fte_mapping"],
    raw["concurrent_projects"],
    raw["oversight_ds_list"],
    raw["ds_list"],
    raw["project_list"],
)
env_kwargs = get_gurobi_env_kwargs()
model = build_gurobi_model(inputs, env_kwargs)
model.setParam(GRB.Param.TimeLimit, 100)
model.optimize()

if model.status in (GRB.OPTIMAL, GRB.TIME_LIMIT):
    baseline_result = {
        "status": "optimal" if model.status == GRB.OPTIMAL else "time_limit",
        "objective_value": model.ObjVal,
        "decision_variables": {v.VarName: v.X for v in model.getVars()},
    }
    with open(OUTPUTS_DIR / "baseline_result.json", "w", encoding="utf-8") as f:
        json.dump(baseline_result, f, indent=2)
    model.write(str(OUTPUTS_DIR / "model.lp"))
    model.write(str(OUTPUTS_DIR / "model.mps"))
    print("Baseline saved to", OUTPUTS_DIR)
else:
    print("Model status:", model.status)

## 3. Build RAG index (once)

Builds `outputs/rag_index/` from the formulation .py, .lp, .mps, and data. Skip if already built.

In [ ]:
from agentic_explain.rag.build_index import build_rag_index

py_path = PROJECT_ROOT / "agentic_explain" / "staffing_model.py"
lp_path = OUTPUTS_DIR / "model.lp"
mps_path = OUTPUTS_DIR / "model.mps"

build_rag_index(
    py_path=py_path,
    lp_path=lp_path if lp_path.exists() else None,
    mps_path=mps_path if mps_path.exists() else None,
    data_dir=DATA_DIR,
    persist_dir=OUTPUTS_DIR / "rag_index",
)
print("RAG index built at", OUTPUTS_DIR / "rag_index")

## 4. Run the workflow

Load baseline and RAG index, then run the agentic workflow for a natural-language query.

In [ ]:
from openai import OpenAI

from agentic_explain.staffing_model import build_gurobi_model
from agentic_explain.rag.build_index import load_rag_index
from agentic_explain.workflow.graph import create_workflow, invoke_workflow

# Load baseline and RAG index
with open(OUTPUTS_DIR / "baseline_result.json", "r", encoding="utf-8") as f:
    baseline_result = json.load(f)

rag_index = load_rag_index(persist_dir=OUTPUTS_DIR / "rag_index")
openai_client = OpenAI()

workflow = create_workflow(
    openai_client=openai_client,
    rag_index=rag_index,
    baseline_result=baseline_result,
    data_dir=str(DATA_DIR),
    build_model_fn=build_gurobi_model,
    inputs=inputs,
    env_kwargs=env_kwargs,
    outputs_dir=str(OUTPUTS_DIR),
)

In [ ]:
# Example query: counterfactual ("why not")
user_query = "Why was Josh not staffed on Ipp IO Pilot in week 6?"

final_state = invoke_workflow(
    workflow,
    user_query,
    baseline_result=baseline_result,
    rag_index=rag_index,
)

print("Query:", user_query)
print()
print("Summary:")
print(final_state.get("final_summary", "(no summary)"))

## 5. Try another query (optional)

In [ ]:
# Direct instruction example
user_query2 = "Force Josh to be staffed on Ipp IO Pilot in week 6"

final_state2 = invoke_workflow(
    workflow,
    user_query2,
    baseline_result=baseline_result,
    rag_index=rag_index,
)

print("Query:", user_query2)
print()
print(final_state2.get("final_summary", "(no summary)"))